<a href="https://colab.research.google.com/github/fpseverino/progetto-ml-ai/blob/main/Severino.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline
import pandas as pd
from sklearn import preprocessing
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import mean_absolute_error
from sklearn.ensemble import BaggingClassifier, BaggingRegressor
from sklearn.ensemble import AdaBoostClassifier, AdaBoostRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import SGDRegressor

In [9]:
class Task:
  def __init__(self, df):
    x = df.drop("Label", axis = 1)
    x = x.drop("Task", axis = 1)
    x = x.drop("Id", axis = 1)
    y = df["Label"]

    self.x_train, self.x_test, self.y_train, self.y_test = train_test_split(x, y, test_size = 0.3)

In [10]:
tasks = []
for i in range(11, 22):
  df = pd.read_csv("Task_{}_clean.csv".format(i))
  task = Task(df)
  tasks.append(task)

In [11]:
bagging = BaggingClassifier(
  KNeighborsClassifier(),
  n_estimators = 10,
  max_samples = 0.5,
  max_features = 0.5
)

In [12]:
for task in tasks:
  scores = cross_val_score(bagging, task.x_train, task.y_train)
  print(scores.mean())

0.711764705882353
0.6830882352941177
0.7470588235294118
0.5426470588235295
0.6507352941176471


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=5.
  warnings.warn(


0.7808823529411765
0.7352941176470589


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(


0.8441176470588235
0.8441176470588235


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(


0.3735294117647059
0.9279411764705883


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(


In [13]:
for task in tasks:
  bagging.fit(task.x_train, task.y_train)
  pred = bagging.predict(task.x_test)

  print(confusion_matrix(task.y_test, pred))
  print(classification_report(task.y_test, pred))

[[ 9  1  0]
 [ 2 14  2]
 [ 2  0  6]]
              precision    recall  f1-score   support

           0       0.69      0.90      0.78        10
           1       0.93      0.78      0.85        18
           2       0.75      0.75      0.75         8

    accuracy                           0.81        36
   macro avg       0.79      0.81      0.79        36
weighted avg       0.83      0.81      0.81        36

[[ 7  5  0]
 [ 0 12  0]
 [ 0  0 12]]
              precision    recall  f1-score   support

           0       1.00      0.58      0.74        12
           1       0.71      1.00      0.83        12
           2       1.00      1.00      1.00        12

    accuracy                           0.86        36
   macro avg       0.90      0.86      0.85        36
weighted avg       0.90      0.86      0.85        36

[[16  1  0]
 [ 5  7  0]
 [ 7  0  0]]
              precision    recall  f1-score   support

           0       0.57      0.94      0.71        17
           1      

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classif

[[ 0  7]
 [ 0 29]]
              precision    recall  f1-score   support

           1       0.00      0.00      0.00         7
           2       0.81      1.00      0.89        29

    accuracy                           0.81        36
   macro avg       0.40      0.50      0.45        36
weighted avg       0.65      0.81      0.72        36

[[17  0  2]
 [ 6  1  2]
 [ 8  0  0]]
              precision    recall  f1-score   support

           0       0.55      0.89      0.68        19
           1       1.00      0.11      0.20         9
           2       0.00      0.00      0.00         8

    accuracy                           0.50        36
   macro avg       0.52      0.34      0.29        36
weighted avg       0.54      0.50      0.41        36

[[ 0  3]
 [ 0 33]]
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         3
           2       0.92      1.00      0.96        33

    accuracy                           0.92        36

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [14]:
bagging = BaggingRegressor(
  SGDRegressor(),
  n_estimators = 1000,
  random_state = 101,
  max_features = 0.8,
  n_jobs = -1
)

for task in tasks:
  bagging.fit(task.x_train, task.y_train)
  print(mean_absolute_error(task.y_test, bagging.predict(task.x_test)))

1.6843166393161836e+20
9.14424434768049e+19
6.182621277329957e+20
7.944547043714495e+19
7.463931445039685e+20
3.1988458513413715e+20
5.449086008249871e+20
1.0467088037461146e+20
1.3340148355723687e+20
5.127433808199631e+20
6.903914478323553e+19


In [15]:
clf = AdaBoostClassifier(n_estimators = 100)

In [16]:
for task in tasks:
  scores = cross_val_score(clf, task.x_train, task.y_train)
  print(scores.mean())

0.8551470588235294
0.9382352941176471
0.8066176470588236
0.6286764705882353
0.47205882352941175


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=5.
  warnings.warn(


0.7323529411764705
0.7095588235294118


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(


0.9154411764705882


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(


0.8676470588235293
0.3625


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(


0.9882352941176471


In [17]:
for task in tasks:
  clf.fit(task.x_train, task.y_train)
  pred = clf.predict(task.x_test)

  print(confusion_matrix(task.y_test, pred))
  print(classification_report(task.y_test, pred))

[[ 0 10  0]
 [ 0 18  0]
 [ 0  0  8]]
              precision    recall  f1-score   support

           0       0.00      0.00      0.00        10
           1       0.64      1.00      0.78        18
           2       1.00      1.00      1.00         8

    accuracy                           0.72        36
   macro avg       0.55      0.67      0.59        36
weighted avg       0.54      0.72      0.61        36



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


[[11  1  0]
 [ 0 12  0]
 [ 0  0 12]]
              precision    recall  f1-score   support

           0       1.00      0.92      0.96        12
           1       0.92      1.00      0.96        12
           2       1.00      1.00      1.00        12

    accuracy                           0.97        36
   macro avg       0.97      0.97      0.97        36
weighted avg       0.97      0.97      0.97        36

[[11  6  0]
 [ 1 11  0]
 [ 0  7  0]]
              precision    recall  f1-score   support

           0       0.92      0.65      0.76        17
           1       0.46      0.92      0.61        12
           2       0.00      0.00      0.00         7

    accuracy                           0.61        36
   macro avg       0.46      0.52      0.46        36
weighted avg       0.59      0.61      0.56        36



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


[[ 0 10  0]
 [ 0 11  0]
 [ 0  0 15]]
              precision    recall  f1-score   support

           0       0.00      0.00      0.00        10
           1       0.52      1.00      0.69        11
           2       1.00      1.00      1.00        15

    accuracy                           0.72        36
   macro avg       0.51      0.67      0.56        36
weighted avg       0.58      0.72      0.63        36



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


[[ 0  5  2]
 [ 0 17  1]
 [ 0  7  4]]
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         7
           1       0.59      0.94      0.72        18
           2       0.57      0.36      0.44        11

    accuracy                           0.58        36
   macro avg       0.39      0.44      0.39        36
weighted avg       0.47      0.58      0.50        36



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


[[ 0  0  1]
 [ 1  5  3]
 [ 0  5 21]]
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         1
           1       0.50      0.56      0.53         9
           2       0.84      0.81      0.82        26

    accuracy                           0.72        36
   macro avg       0.45      0.45      0.45        36
weighted avg       0.73      0.72      0.73        36

[[ 5  2]
 [ 2 27]]
              precision    recall  f1-score   support

           1       0.71      0.71      0.71         7
           2       0.93      0.93      0.93        29

    accuracy                           0.89        36
   macro avg       0.82      0.82      0.82        36
weighted avg       0.89      0.89      0.89        36

[[ 0  1]
 [ 1 34]]
              precision    recall  f1-score   support

           1       0.00      0.00      0.00         1
           2       0.97      0.97      0.97        35

    accuracy                           0.94        36

In [20]:
booster = AdaBoostRegressor(
  SGDRegressor(),
  random_state = 101,
  n_estimators = 100
)

for task in tasks:
  booster.fit(task.x_train, task.y_train)
  print(mean_absolute_error(task.y_test, booster.predict(task.x_test)))

2.840075224494329e+20
1.3829334367020717e+20
1.046072215018156e+21
4.298934219395467e+20
9.633364371267094e+21
1.877410961741324e+21
3.0685535492900184e+21
1.1662907991933855e+21
1.7669587440433438e+21
4.808959341340898e+21
4.058082432004067e+20


In [21]:
regr = GradientBoostingRegressor(
  n_estimators = 500,
  learning_rate = 0.01,
  random_state = 101
)

for task in tasks:
  regr.fit(task.x_train, task.y_train)
  print(mean_absolute_error(task.y_test, regr.predict(task.x_test)))

0.08513074110665717
0.032450561065974354
0.32006980545266617
0.08634231589563628
0.42051871813842634
0.25050991663137734
0.18218728278578877
0.0806588321837521
0.17902518457077088
0.6452622664253853
0.02443669184111312
